# Getting Started

Go to the setupTable and input your unique token

DO NOT USE MY KEY (Blake)

## Run cells through initialize player

In [6]:
import requests
import time
import json
import hashlib
from timeit import default_timer as timer

In [7]:
class Stack():
    def __init__(self):
        self.stack = []
    def push(self, value):
        self.stack.append(value)
    def pop(self):
        if self.size() > 0:
            return self.stack.pop()
        else:
            return None
    def size(self):
        return len(self.stack)

In [8]:
class Queue():
    def __init__(self):
        self.queue = []
    def enqueue(self, value):
        self.queue.append(value)
    def dequeue(self):
        if self.size() > 0:
            return self.queue.pop(0)
        else:
            return None
    def size(self):
        return len(self.queue)

In [9]:
# Import map and room info
with open('worldMap.json', 'r') as f:
    worldMap = json.load(f)
    
with open('worldRoomIndex.json', 'r') as f:
    worldRoomIndex = json.load(f)

In [10]:
# Utils
inverseMap = {'n':'s', 's':'n', 'e':'w', 'w':'e'}

## Input unique token key here

In [475]:
setupTable = {
    "url": 'https://lambda-treasure-hunt.herokuapp.com/api/',
    "token": "d2679616e63cea70c89ff019addc2677a27c9388",
    "init": 'adv/init/',
    "move": 'adv/move/',
    "take": 'adv/take/',
    "sell": 'adv/sell/',
    "drop": 'adv/drop/',
    "checkStatus": 'adv/status/',
    "examine": 'adv/examine/',
    "wearItem": 'adv/wear/',
    "removeItem": 'adv/undress/',
    "nameChange": 'adv/change_name/',
    "pray": 'adv/pray/',
    "fly": 'adv/fly/',
    "dash": 'adv/dash/',
    "companionLetCarry": 'adv/carry/',
    "companionReceiveBack":  'adv/receive/',
    "lambdaCoinProof": 'bc/last_proof/',
    "lambdaCoinMine": 'bc/mine/',
    "transmogrify": 'adv/transmogrify'
    
}
header={'Authorization': 'Token ' + setupTable['token']}
init = setupTable['url'] + setupTable['init']
move = setupTable['url'] + setupTable['move']
take = setupTable['url'] + setupTable['take']
sell = setupTable['url'] + setupTable['sell']
drop = setupTable['url'] + setupTable['drop']
checkStatus = setupTable['url'] + setupTable['checkStatus']
examine = setupTable['url'] + setupTable['examine']
wearItem = setupTable['url'] + setupTable['wearItem']
removeItem = setupTable['url'] + setupTable['removeItem']
nameChange = setupTable['url'] + setupTable['nameChange']
pray = setupTable['url'] + setupTable['pray']
fly = setupTable['url'] + setupTable['fly']
dash = setupTable['url'] + setupTable['dash']
companionLetCarry = setupTable['url'] + setupTable['companionLetCarry']
companionReceiveBack = setupTable['url'] + setupTable['companionReceiveBack']
lambdaCoinProof = setupTable['url'] + setupTable['lambdaCoinProof']
lambdaCoinMine = setupTable['url'] + setupTable['lambdaCoinMine']
transmogrify = setupTable['url'] + setupTable['transmogrify']

In [476]:
class Player:
    
    def __init__(self):
        self.initResponse = requests.get(url=init, headers=header)
        self.playersMap = worldMap
        self.coolDown = None
        self.status = None
        self.inventory = None
        
        self.currentRoomID = None
        self.previousRoomID = None
        self.currentRoomInfo = None
        self.lastMoveDirection = None

        # Blockchain 
        self.lastProof = None
        self.validProof = None

        
    def getRoomInfo(self):
        time.sleep(1)
        response = requests.get(url=init, headers=header)
        
        if response.status_code == 200:
            return response.json()
        else:
            print(response.json())
            print(f'400 status code at getRoomInfo method {response.status_code}')
            
    def assignCurrentRoomInfo(self):
        self.currentRoomInfo = self.getRoomInfo()
        
    def assignCurrentRoomID(self):
        self.currentRoomID = self.getCurrentRoomID()
            
    def getCurrentRoomID(self):
        return self.currentRoomInfo['room_id']
            
    def writeNewRoomToMap(self):
        exits = self.currentRoomInfo['exits']
        
        if self.currentRoomID not in self.playersMap:
            self.playersMap[self.currentRoomID] = {key: '?' for key in exits}
            
    def writeMoveDirectionsForCurrentAndPreviousRoom(self, previousRoomID, currentRoomID, direction):
        self.playersMap[previousRoomID][direction] = self.currentRoomID
        self.playersMap[currentRoomID][inverseMap[direction]] = self.previousRoomID
        
    def moveRoom(self, direction):
        if str(self.currentRoomID) in self.playersMap and self.playersMap[str(self.currentRoomID)][direction] !='?':
            print("Wise Explore")
            
            self.previousRoomID = self.currentRoomID
            
            nextRoom = self.playersMap[str(self.currentRoomID)][direction]
            
            postRequestBody = {"direction":direction, "next_room_id":str(nextRoom)}
            response = requests.post(url=move, headers=header, json=postRequestBody)
            self.currentRoomInfo = response.json()
            self.coolDown = response.json()['cooldown']
            
            time.sleep(self.coolDown)
            self.currentRoomID = self.getCurrentRoomID()
            self.lastMoveDirection = direction
            
            print("Leaving room: "+str(player.previousRoomID)+" entering room: "+str(player.currentRoomID))
            
        else:
            print('Normal Explore')
            self.previousRoomID = self.currentRoomID
            
            postRequestBody = {"direction":direction}
            response = requests.post(url=move, headers=header, json=postRequestBody)

            self.currentRoomInfo = response.json()
            self.currentRoomID = self.getCurrentRoomID()
            
            if response.status_code == 200:
                self.coolDown = response.json()['cooldown']
                time.sleep(self.coolDown)
                
                self.lastMoveDirection = direction
                
                self.writeNewRoomToMap()
                self.writeMoveDirectionsForCurrentAndPreviousRoom(self.previousRoomID, self.currentRoomID, direction)
                print("Leaving room: "+str(player.previousRoomID)+" entering room: "+str(player.currentRoomID))
            else:
                print("The move was not a success")
                
        return self.getRoomInfo()
    
    def findUnexploredRoom(self):
        for direction in self.playersMap[self.currentRoomID]:
            if self.playersMap[self.currentRoomID][direction] == '?':
                return direction
        return direction
    
    def takeItem(self):
        itemName = self.currentRoomInfo['items']
        for items in itemName:
            postRequestBody = {"name": items}
            response = requests.post(url=take, headers=header, json=postRequestBody)
            
            self.coolDown = response.json()['cooldown']
            print("Cooling down for :"+ str(self.coolDown)+"seconds")
            time.sleep(self.coolDown)
        print("You have picked up ", itemName)
        
    def dropItem(self, itemName):
        pass
    
    def sellItem(self):
        self.checkPlayerStatus()
        time.sleep(2)
        if self.inventory:
            for item in self.inventory:
                postRequestBody = {"name": item, "confirm":"yes"}
                response = requests.post(url=sell, headers=header, json=postRequestBody)
                self.coolDown = response.json()['cooldown']
                time.sleep(self.coolDown)
                print("You have sold: ", item)
                
    def changeName(self):
        postRequestBody = {"name": "[CPHACHE]", "confirm":"aye"}
        response= requests.post(url=nameChange, headers=header, json=postRequestBody)
        
        print(response)
        print(response.json())
        
    def prayToShrine(self):
        response = requests.post(url=pray, headers=header)
        print(response.json())
        self.coolDown = response.json()['cooldown']
        time.sleep(self.coolDown)
        
    def getLambdaCoinProof(self):
        response = requests.get(url=lambdaCoinProof, headers=header)
        self.lastProof = response.json()
        return response.json()
    
    def mineCoin(self):
        newProof = self.validProof
        postRequestBody = {"proof":newProof}
        response = requests.post(url=lambdaCoinMine, headers=header, json=postRequestBody)
        print(response)
        return response.json()
        
    def examineWell(self):
        postRequestBody = {"name":"WELL"}
        response = requests.post(url=examine, headers=header, json=postRequestBody)
        return response.json()
        
    def transmogrifyItem(self, itemName):
        postRequestBody = {"name":itemName}
        response = requests.post(url=transmogrify, headers=header, json=postRequestBody)
        return response.json()
    
    def checkPlayerStatus(self):
        response = requests.post(url=checkStatus, headers=header)
        self.status = response.json()
        self.coolDown = response.json()['cooldown']
        
        if response.json()['inventory']:
            self.inventory = response.json()['inventory']
            
        time.sleep(self.coolDown)
        
        print(response.json())
        
        

In [477]:
def exploreWorld(player):
    """Traverses the world and records rooms to playersMap"""
    print('Initializing...')
    
    roomStack = Stack()
    moveStack = Stack()
    count = 0
    

    player.currentRoomInfo = player.getRoomInfo()
    print('Loading currentRoomInfo')
    time.sleep(5)
    
    player.currentRoomID = player.currentRoomInfo['room_id']
    player.writeNewRoomToMap()
    
    roomStack.push(player.currentRoomID)
    
    roomDict = {}
    roomDict[player.currentRoomID] = player.currentRoomInfo
    
    while roomStack.size() > 0:
        
        direction = player.findUnexploredRoom()
        print("Direction", direction)
        
        if player.playersMap[player.currentRoomID][direction] == '?':
            
            player.moveRoom(direction)
            moveStack.push(direction)
            
            print("Leaving room: "+str(player.previousRoomID)+" entering room: "+str(player.currentRoomID))
            
            roomStack.push(player.currentRoomID)
            
            if player.currentRoomID not in roomDict:
                roomDict[player.currentRoomID] = player.currentRoomInfo
                
        else:
            print("roomStack:", roomStack.stack)
            print('moveStack:', moveStack.stack)
            roomStack.pop()
            lastMove = moveStack.stack[-1]
            backTrackingDirection = inverseMap[lastMove]
            
            if roomStack.size() > 0:
                player.previousRoomID = player.currentRoomID
                player.moveRoom(backTrackingDirection)
                player.lastMoveDirection = moveStack.stack[-1]      

            moveStack.pop()

    return roomDict          

In [478]:
def bfs(player, starting_vertex, destination_vertex):
    """
    Return a list containing the shortest path from
    starting_vertex to destination_vertex in
    breath-first order.
    """

    # Create an empty queue and enqueue the starting vertex ID
    pathQueue = Queue()
    directionQueue = Queue()
    
    pathQueue.enqueue([str(starting_vertex)])
    directionQueue.enqueue(['Starting Location'])

    # Create a Set to store visited vertices
    visited = set()

    # While the queue is not empty...
    while destination_vertex not in visited:
        # Dequeue the first vertex
        path = pathQueue.dequeue()
        directionPath = directionQueue.dequeue()

        # If that vertex has not been visited
        v = path[-1]
        
        if v not in visited:
            if v == destination_vertex:
                return path, directionPath
            visited.add(v)
            
        for direction, neighboringRoom in player.playersMap[v].items():
            path_copy = list(path)
            path_copy.append(str(neighboringRoom))
            pathQueue.enqueue(path_copy)
            
            directionCopy = list(directionPath)
            directionCopy.append(direction)
            directionQueue.enqueue(directionCopy)

    return None

In [479]:
def dfs(player, starting_vertex, destination_vertex):
    """
    Return a list containing the shortest path from
    starting_vertex to destination_vertex in
    breath-first order.
    """

    # Create an empty queue and enqueue the starting vertex ID
    pathStack = Stack()
    directionStack = Stack()
    
    pathStack.push([str(starting_vertex)])
    directionStack.push(['Starting Location'])

    # Create a Set to store visited vertices
    visited = set()

    # While the queue is not empty...
    while destination_vertex not in visited:
        # Dequeue the first vertex
        path = pathStack.pop()
        directionPath = directionStack.pop()

        # If that vertex has not been visited
        v = path[-1]
        
        if v not in visited:
            if v == destination_vertex:
                return path, directionPath
            visited.add(v)
            
        for direction, neighboringRoom in player.playersMap[v].items():
            path_copy = list(path)
            path_copy.append(str(neighboringRoom))
            pathStack.push(path_copy)
            
            directionCopy = list(directionPath)
            directionCopy.append(direction)
            directionStack.push(directionCopy)

    return None

In [480]:
def moveToSpecificRoom(player, starting_vertex, destination_vertex):
    path, directionPath = bfs(player, starting_vertex, destination_vertex)
    
    print(path)
    directions = directionPath[1:]
    print(directions)
    for direction in directions:
        player.moveRoom(direction)
        time.sleep(player.coolDown)
        print(player.currentRoomInfo)
        
    print("You have reached your destination")

In [481]:
def proof_of_work(player):
    start = timer()
    
    last_proof = player.lastProof['proof']
    
    
    print("Searching for next proof")
    #block_string = json.dumps(last_proof, sort_keys=True)
    proof = 0
    #  TODO: Your code here
    while valid_proof(last_proof, proof) is False:
        proof +=1
    print("Proof found: " + str(proof) + " in " + str(timer() - start))
    player.validProof = proof
    return proof


def valid_proof(last_hash, proof):
    
    difficulty = player.lastProof['difficulty']
    leadingZeros = '0'* difficulty
    
    guess = f'{last_hash}{proof}'.encode()
    guess_hash = hashlib.sha256(guess).hexdigest()
    if str(guess_hash)[:difficulty] == leadingZeros:
        print(guess_hash)
        return True
    else:
        return False

## Initialize Player

In [482]:
player = Player()
player.assignCurrentRoomInfo()
player.assignCurrentRoomID()

In [468]:
player.currentRoomID

495

## Move to Specificied Room

In [459]:
moveToSpecificRoom(player, player.currentRoomID, '495')

['276', '197', '232', '235', '330', '383', '495']
['e', 'n', 'w', 'n', 'w', 'w']
Wise Explore
Leaving room: 276 entering room: 197
{'room_id': 197, 'title': 'A misty room', 'description': 'You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction.', 'coordinates': '(53,56)', 'elevation': 0, 'terrain': 'NORMAL', 'players': [], 'items': [], 'exits': ['n', 'e', 'w'], 'cooldown': 7.5, 'errors': [], 'messages': ['You have walked east.', 'Wise Explorer: -50% CD']}
Wise Explore
Leaving room: 197 entering room: 232
{'room_id': 232, 'title': 'A misty room', 'description': 'You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction.', 'coordinates': '(53,57)', 'elevation': 0, 'terrain': 'NORMAL', 'players': [], 'items': [], 'exits': ['n', 's', 'w'], 'cooldown': 7.5, 'errors': [], 'messages': ['You have walked north.', 'Wise Explorer: -50% CD']}
Wise Explore
Leaving room: 232 entering room: 235
{'

In [483]:
itemName = player.status['inventory'][-1]

TypeError: 'NoneType' object is not subscriptable

In [473]:
itemName

'shiny treasure'

In [485]:
player.transmogrifyItem(itemName)

{'room_id': 495,
 'title': 'The Transmogriphier',
 'description': 'A strange machine stands in this room.  There is a large opening on the top.  A placard reads, "Test your luck!  One item and one Lambdacoin!"',
 'coordinates': '(50,58)',
 'elevation': 0,
 'terrain': 'NORMAL',
 'players': ['PWalis', 'justinchen32'],
 'items': [],
 'exits': ['e'],
 'cooldown': 15.0,
 'errors': [],
 'messages': ['Your shiny treasure transmogrified into nice jacket!']}

In [486]:
player.checkPlayerStatus()

{'name': '[CPHACHE]', 'cooldown': 1.0, 'encumbrance': 8, 'strength': 10, 'speed': 10, 'gold': 200, 'bodywear': None, 'footwear': None, 'inventory': ['nice jacket', 'tiny treasure', 'small treasure', 'tiny treasure', 'small treasure', 'tiny treasure'], 'abilities': ['pray', 'mine', 'fly', 'dash', 'carry'], 'status': [], 'has_mined': True, 'errors': [], 'messages': []}


## BlockChain

In [411]:
player.getLambdaCoinProof()
proof_of_work(player)
player.mineCoin()

Searching for next proof
0000007edc8a0e76c7ba6c7deff8314d129b0d51e37843af01284f66ea1054e1
Proof found: 15596417 in 45.538837959000375
<Response [200]>


{'name': '[CPHACHE]',
 'description': '[CPHACHE] looks like an ordinary person.',
 'cooldown': 115.0,
 'errors': ['There is no coin here: +100s'],
 'messages': []}

## Examine Well

In [448]:
player.examineWell()

{'name': 'Wishing Well',
 'description': 'You see a faint pattern in the water...\n\n10000010\n00000001\n01001101\n01001000\n00000001\n10000010\n00000001\n01101001\n01001000\n00000001\n10000010\n00000001\n01101110\n01001000\n00000001\n10000010\n00000001\n01100101\n01001000\n00000001\n10000010\n00000001\n00100000\n01001000\n00000001\n10000010\n00000001\n01111001\n01001000\n00000001\n10000010\n00000001\n01101111\n01001000\n00000001\n10000010\n00000001\n01110101\n01001000\n00000001\n10000010\n00000001\n01110010\n01001000\n00000001\n10000010\n00000001\n00100000\n01001000\n00000001\n10000010\n00000001\n01100011\n01001000\n00000001\n10000010\n00000001\n01101111\n01001000\n00000001\n10000010\n00000001\n01101001\n01001000\n00000001\n10000010\n00000001\n01101110\n01001000\n00000001\n10000010\n00000001\n00100000\n01001000\n00000001\n10000010\n00000001\n01101001\n01001000\n00000001\n10000010\n00000001\n01101110\n01001000\n00000001\n10000010\n00000001\n00100000\n01001000\n00000001\n10000010\n00000

## Dash

In [430]:
direction = 's'
nextRoomIDs = []
for room in range(5)
    player.playersMap[str(player.currentRoomID)]

{'n': 227, 's': 147, 'e': 206}

In [436]:
postRequestBody = {"direction":"w",
                   "num_rooms": "3",
                   "next_room_ids": "30,27,20",
                  }
response = requests.post(url=dash, headers=header, json=postRequestBody)
print(response.json())

{'room_id': 20, 'title': 'A misty room', 'description': 'You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction.', 'coordinates': '(60,63)', 'elevation': 0, 'terrain': 'NORMAL', 'players': ['Eric_F'], 'items': [], 'exits': ['n', 's', 'e', 'w'], 'cooldown': 16.5, 'errors': [], 'messages': ['You have dashed to room 30.', 'You have dashed to room 27.', 'You have dashed to room 20.']}
